In [ ]:
!pip install -U datasets
!pip install gcsfs==2025.3.0 fsspec==2025.3.0
!pip install transformers accelerate peft bitsandbytes evaluate rouge-score nltk wandb
!pip install -U bert_scor
!pip show transformers

ERROR: Could not find a version that satisfies the requirement bert_scor (from versions: none)
ERROR: No matching distribution found for bert_scor
Name: transformers
Version: 4.51.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
!pip install -U datasets

In [4]:
!pip install evaluate rouge_score accelerate peft bitsandbytes evaluate rouge-score nltk wandb bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [34]:
from google.colab import userdata
GEMINI_KEY=userdata.get('GEMINI_KEY')

### Gemini

In [2]:
import google.generativeai as genai
import os
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datasets import load_dataset
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
import time
nltk.download('punkt')
nltk.download('punkt_tab')

# Налаштування API ключа для Gemini
genai.configure(api_key=GEMINI_KEY)

# Завантаження валідаційного датасету
val_dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation[:50]")  # Обмежуємо до 50 прикладів

# Завантаження метрик
rouge_metric = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

# Функція для обчислення BLEU
def calculate_bleu(predictions, references):
    smoothie = SmoothingFunction().method1
    bleu_scores = []

    for pred, ref in zip(predictions, references):
        pred_tokens = nltk.word_tokenize(pred.lower())
        ref_tokens = [nltk.word_tokenize(ref.lower())]
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)
        bleu_scores.append(score)

    return np.mean(bleu_scores)

# Функція для zero-shot запиту до Gemini
def gemini_zero_shot_summarize(article):
    try:
        # Створюємо prompt для сумаризації статті
        prompt = f"""Summarize the following news article concisely:

Article:
{article}

Summary:"""

        # Отримуємо модель і виконуємо запит
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content(prompt)

        # Повертаємо текст відповіді
        return response.text.strip()
    except Exception as e:
        print(f"Error with Gemini API: {e}")
        # У випадку помилки повертаємо порожній рядок
        return "API Error occurred"

# Функція для few-shot запиту до Gemini
def gemini_few_shot_summarize(article, examples=3):
    try:
        # Формуємо приклади для few-shot навчання
        few_shot_examples = ""
        for i in range(examples):
            example = train_dataset[i]
            few_shot_examples += f"""Article: {example['article'][:300]}...
Summary: {example['highlights']}

"""

        # Створюємо prompt з прикладами та новою статтею
        prompt = f"""I'm going to give you examples of news article summarization, then ask you to summarize a new article.

{few_shot_examples}
Now, summarize this article:

Article: {article}

Summary:"""

        # Отримуємо модель і виконуємо запит
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content(prompt)

        # Повертаємо текст відповіді
        return response.text.strip()
    except Exception as e:
        print(f"Error with Gemini API: {e}")
        # У випадку помилки повертаємо порожній рядок
        return "API Error occurred"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Завантажуємо датасет для few-shot прикладів
train_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:10]")

# Zero-shot експеримент
print("Проведення zero-shot експерименту з Gemini...")
zero_shot_results = []

for i, example in enumerate(tqdm(val_dataset, desc="Zero-shot Gemini")):
    article = example["article"]
    reference = example["highlights"]

    # Отримуємо сумаризацію через Gemini API
    summary = gemini_zero_shot_summarize(article)

    # Додаємо в результати
    zero_shot_results.append({
        "article": article,
        "reference": reference,
        "gemini_zero_shot": summary
    })

    # Невелика затримка, щоб не перевантажувати API
    time.sleep(5)

In [26]:
# Few-shot експеримент
print("Проведення few-shot експерименту з Gemini...")
few_shot_results = []

for i, example in enumerate(tqdm(val_dataset, desc="Few-shot Gemini")):
    article = example["article"]
    reference = example["highlights"]

    # Отримуємо сумаризацію через Gemini API з few-shot прикладами
    summary = gemini_few_shot_summarize(article, examples=3)

    # Додаємо в результати
    few_shot_results.append({
        "article": article,
        "reference": reference,
        "gemini_few_shot": summary
    })

    # Невелика затримка, щоб не перевантажувати API
    time.sleep(5)

Проведення few-shot експерименту з Gemini...


Few-shot Gemini:   0%|          | 0/50 [00:00<?, ?it/s]

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 379.88ms


In [27]:
zero_shot_results[0]

{'article': '(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don\'t know, but the fact that so many people can have a life extension, that\'s pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I\'m just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard\'s gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, according to California Pacifi

In [28]:
few_shot_results[0]


{'article': '(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don\'t know, but the fact that so many people can have a life extension, that\'s pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I\'m just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard\'s gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, according to California Pacifi

In [31]:
# Об'єднуємо результати
combined_results = []
for i in range(len(zero_shot_results)):
    combined_results.append({
        "article": zero_shot_results[i]["article"],
        "reference": zero_shot_results[i]["reference"],
        "gemini_zero_shot": zero_shot_results[i]["gemini_zero_shot"],
        "gemini_few_shot": few_shot_results[i]["gemini_few_shot"]
    })

# Зберігаємо результати в CSV файл
results_df = pd.DataFrame(combined_results)
results_df.to_csv("gemini_summarization_results.csv", index=False)
print("Результати збережено в gemini_summarization_results.csv")

# Обчислюємо метрики для zero-shot
zero_shot_preds = [item["gemini_zero_shot"] for item in combined_results]
references = [item["reference"] for item in combined_results]

zero_shot_rouge = rouge_metric.compute(
    predictions=zero_shot_preds,
    references=references,
    use_stemmer=True
)

zero_shot_bertscore = bertscore.compute(
    predictions=zero_shot_preds,
    references=references,
    lang="en"
)

zero_shot_bleu = calculate_bleu(zero_shot_preds, references)

zero_shot_metrics = {
    "rouge1": round(zero_shot_rouge["rouge1"], 4),
    "rouge2": round(zero_shot_rouge["rouge2"], 4),
    "rougeL": round(zero_shot_rouge["rougeL"], 4),
    "bertscore_f1": round(np.mean(zero_shot_bertscore["f1"]), 4),
    "bleu": round(zero_shot_bleu, 4)
}

print("Zero-shot Gemini метрики:")
print(zero_shot_metrics)

# Обчислюємо метрики для few-shot
few_shot_preds = [item["gemini_few_shot"] for item in combined_results]

few_shot_rouge = rouge_metric.compute(
    predictions=few_shot_preds,
    references=references,
    use_stemmer=True
)

few_shot_bertscore = bertscore.compute(
    predictions=few_shot_preds,
    references=references,
    lang="en"
)

few_shot_bleu = calculate_bleu(few_shot_preds, references)

few_shot_metrics = {
    "rouge1": round(few_shot_rouge["rouge1"], 4),
    "rouge2": round(few_shot_rouge["rouge2"], 4),
    "rougeL": round(few_shot_rouge["rougeL"], 4),
    "bertscore_f1": round(np.mean(few_shot_bertscore["f1"]), 4),
    "bleu": round(few_shot_bleu, 4)
}

print("Few-shot Gemini метрики:")
print(few_shot_metrics)

Результати збережено в gemini_summarization_results.csv
Zero-shot Gemini метрики:
{'rouge1': np.float64(0.3374), 'rouge2': np.float64(0.1155), 'rougeL': np.float64(0.2267), 'bertscore_f1': np.float64(0.8746), 'bleu': np.float64(0.0502)}
Few-shot Gemini метрики:
{'rouge1': np.float64(0.3249), 'rouge2': np.float64(0.1178), 'rougeL': np.float64(0.2213), 'bertscore_f1': np.float64(0.8716), 'bleu': np.float64(0.0531)}


### Маленька decoder-only LLM модель

In [85]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    TrainerCallback
)
from transformers import DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
from transformers.trainer_utils import EvalPrediction
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import wandb
import numpy as np
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Ініціалізація wandb
wandb.init(project="falcon-lora-summarization", name="falcon_rw_1b_experiment")

# 1. Завантаження датасетів
train_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1000]")
eval_dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation[:100]")

# 2. Токенайзер та модель
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16
)

# 3. LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.09,
    bias="none",
    target_modules=["query_key_value"]  # для Falcon
)
model = get_peft_model(model, peft_config)

# 4. Препроцесінг
def preprocess(example):
    prompt = "Summarize the following news article:\n" + example["article"] + "\nSummary:"
    target = example["highlights"]
    full = prompt + " " + target

    tokenized = tokenizer(full, max_length=640, truncation=True, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

train_dataset = train_dataset.map(preprocess)
eval_dataset = eval_dataset.map(preprocess, remove_columns=['article', 'highlights', 'id'])

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# 5. Метрики
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def compute_metrics(eval_pred: EvalPrediction):
    preds = tokenizer.batch_decode(eval_pred.predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(eval_pred.label_ids, skip_special_tokens=True)

    # ініціалізація метрик
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []

    for l, p in zip(labels, preds):
        rouge_results = scorer.score(l, p)
        rouge1_scores.append(rouge_results["rouge1"].fmeasure)
        rouge2_scores.append(rouge_results["rouge2"].fmeasure)
        rougeL_scores.append(rouge_results["rougeL"].fmeasure)

        bleu_score = sentence_bleu([l.split()], p.split())
        bleu_scores.append(bleu_score)

    # BERTScore
    P, R, F1 = bert_score(preds, labels, lang="en", verbose=False)

    results = {
        "rouge1": round(np.mean(rouge1_scores), 4),
        "rouge2": round(np.mean(rouge2_scores), 4),
        "rougeL": round(np.mean(rougeL_scores), 4),
        "bertscore_f1": round(np.mean(F1.numpy()), 4),
        "bleu": round(np.mean(bleu_scores), 4)
    }

    return results

class BatchEvalCallback(TrainerCallback):
    def __init__(self, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer

    def on_evaluate(self, args, state, control, model=None, **kwargs):
        print("Running batch-wise evaluation to avoid OOM...")
        model.eval()
        eval_dataloader = DataLoader(eval_dataset, batch_size=4, collate_fn=collate_fn)
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []
        bleu_scores = []
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in tqdm(eval_dataloader, desc="Evaluating"):
                input_ids = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)

                generated_ids = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=128
                )

                preds = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
                labels = self.tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

                all_preds.extend(preds)
                all_labels.extend(labels)

                for p, l in zip(preds, labels):
                    rouge_results = scorer.score(l, p)
                    rouge1_scores.append(rouge_results["rouge1"].fmeasure)
                    rouge2_scores.append(rouge_results["rouge2"].fmeasure)
                    rougeL_scores.append(rouge_results["rougeL"].fmeasure)

                    bleu = sentence_bleu([l.split()], p.split())
                    bleu_scores.append(bleu)

        P, R, F1 = bert_score(all_preds, all_labels, lang="en", verbose=False)

        rouge1_avg = np.mean(rouge1_scores)
        rouge2_avg = np.mean(rouge2_scores)
        rougeL_avg = np.mean(rougeL_scores)
        bleu_avg = np.mean(bleu_scores)
        bertscore_f1_avg = np.mean(F1.numpy())

        print(f"[Eval] ROUGE-1: {rouge1_avg:.4f}, ROUGE-2: {rouge2_avg:.4f}, ROUGE-L: {rougeL_avg:.4f}")
        print(f"[Eval] BERTScore F1: {bertscore_f1_avg:.4f}, BLEU: {bleu_avg:.4f}")

        # Логування wandb
        wandb.log({
            "eval/rouge1": rouge1_avg,
            "eval/rouge2": rouge2_avg,
            "eval/rougeL": rougeL_avg,
            "eval/bertscore_f1": bertscore_f1_avg,
            "eval/bleu": bleu_avg,
            "step": state.global_step
        })

# 7. Аргументи навчання
training_args = TrainingArguments(
    output_dir="./falcon_lora_summarization",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    logging_steps=2,
    save_strategy="epoch",
    report_to="wandb",
    eval_strategy="epoch",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    callbacks=[BatchEvalCallback(tokenizer)],
)

eval/loss,▁
eval/model_preparation_time,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/loss,3.64635
eval/model_preparation_time,0.0019
eval/runtime,9.3901
eval/samples_per_second,6.816
eval/steps_per_second,3.408


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [86]:
# Оцінка метрик до файнтюнінгу
print("Оцінка метрик до файнтюнінгу...")
eval_before_results = trainer.evaluate()
print("Метрики до файнтюнінгу:")
print(eval_before_results)

Оцінка метрик до файнтюнінгу...


Running batch-wise evaluation to avoid OOM...


Evaluating: 100%|██████████| 25/25 [05:49<00:00, 13.97s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Eval] ROUGE-1: 0.8808, ROUGE-2: 0.8804, ROUGE-L: 0.8808
[Eval] BERTScore F1: 0.9919, BLEU: 0.7889
Метрики до файнтюнінгу:
{'eval_loss': 3.3749420642852783, 'eval_model_preparation_time': 0.0018, 'eval_runtime': 14.3414, 'eval_samples_per_second': 6.973, 'eval_steps_per_second': 3.486}


In [87]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.154600,2.001407,0.001800
2,1.819100,2.006734,0.001800
3,1.951200,2.010090,0.001800


Running batch-wise evaluation to avoid OOM...


Evaluating: 100%|██████████| 25/25 [05:41<00:00, 13.64s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Eval] ROUGE-1: 0.9187, ROUGE-2: 0.9185, ROUGE-L: 0.9187
[Eval] BERTScore F1: 0.9863, BLEU: 0.8488
Running batch-wise evaluation to avoid OOM...


Evaluating: 100%|██████████| 25/25 [05:42<00:00, 13.69s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Eval] ROUGE-1: 0.9186, ROUGE-2: 0.9184, ROUGE-L: 0.9186
[Eval] BERTScore F1: 0.9863, BLEU: 0.8483
Running batch-wise evaluation to avoid OOM...


Evaluating: 100%|██████████| 25/25 [05:41<00:00, 13.66s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Eval] ROUGE-1: 0.9194, ROUGE-2: 0.9192, ROUGE-L: 0.9194
[Eval] BERTScore F1: 0.9863, BLEU: 0.8498


TrainOutput(global_step=375, training_loss=1.9872185242970783, metrics={'train_runtime': 2059.8046, 'train_samples_per_second': 1.456, 'train_steps_per_second': 0.182, 'total_flos': 1.394122162176e+16, 'train_loss': 1.9872185242970783, 'epoch': 3.0})

In [88]:
# Оцінка метрик після файнтюнінгу
print("Оцінка метрик після файнтюнінгу...")
eval_results = trainer.evaluate()
print("Метрики після файнтюнінгу:")
print(eval_results)

Оцінка метрик після файнтюнінгу...


Running batch-wise evaluation to avoid OOM...


Evaluating: 100%|██████████| 25/25 [05:44<00:00, 13.79s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Eval] ROUGE-1: 0.9194, ROUGE-2: 0.9192, ROUGE-L: 0.9194
[Eval] BERTScore F1: 0.9863, BLEU: 0.8498
Метрики після файнтюнінгу:
{'eval_loss': 2.010089635848999, 'eval_model_preparation_time': 0.0018, 'eval_runtime': 15.1939, 'eval_samples_per_second': 6.582, 'eval_steps_per_second': 3.291, 'epoch': 3.0}


In [ ]:
# Логуємо підсумкові результати в wandb
wandb.log({
    "final/rouge1": eval_results.get("eval_rouge1", 0),
    "final/rouge2": eval_results.get("eval_rouge2", 0),
    "final/rougeL": eval_results.get("eval_rougeL", 0),
    "final/bertscore_f1": eval_results.get("eval_bertscore_f1", 0),
    "final/bleu": eval_results.get("eval_bleu", 0)
})

In [89]:
for item_before, item_after in zip(eval_before_results.items(), eval_results.items()):
  print(f'{item_before[0]}')
  print(f'before fine-tune: {item_before[1]}')
  print(f'after fine-tune: {item_after[1]}')
  print('--------')

eval_loss
before fine-tune: 3.3749420642852783
after fine-tune: 2.010089635848999
--------
eval_model_preparation_time
before fine-tune: 0.0018
after fine-tune: 0.0018
--------
eval_runtime
before fine-tune: 14.3414
after fine-tune: 15.1939
--------
eval_samples_per_second
before fine-tune: 6.973
after fine-tune: 6.582
--------
eval_steps_per_second
before fine-tune: 3.486
after fine-tune: 3.291
--------


eval_rouge1
before fine-tune: 0.8808
after fine-tune: 0.9194

eval_rouge2
before fine-tune: 0.8804
after fine-tune: 0.9192

eval_rougeL
before fine-tune: 0.8808
after fine-tune: 0.9194

eval_bertscore_f1
before fine-tune: 0.9919
after fine-tune: 0.9863

eval_bleu
before fine-tune: 0.7889
after fine-tune: 0.8498



###### Приклад до файн-тюну

In [104]:
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16
)

# The input article
input_text = few_shot_results[0]['article']

# Prepare prompt for the model
prompt = "Summarize the following news article:\n" + input_text + "\nSummary:"

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(model.device)

# Generate the summary
model.eval()
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )

# Decode the output
summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print result
print("Summary:")
print(summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary:
Summarize the following news article:
(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I'm just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard's gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, 

###### Приклад після файн-тюну

In [107]:
model_name = "/content/falcon_lora_summarization/checkpoint-375"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16
)

# The input article
input_text = few_shot_results[0]['article']

# Prepare prompt for the model
prompt = "Summarize the following news article:\n" + input_text + "\nSummary:"

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(model.device)

# Generate the summary
model.eval()
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )

# Decode the output
summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print result
print("Summary:")
print(summary)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary:
Summarize the following news article:
(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I'm just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard's gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, 

### Модель T5

In [36]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import numpy as np
import wandb
from tqdm.auto import tqdm
import evaluate
from transformers.trainer_callback import TrainerCallback
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

# Ініціалізація wandb
wandb.init(project="t5-summarization", name="t5_small_cnn_dailymail_with_extra_metrics")

# Завантаження датасету
train_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1000]")
val_dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation[:100]")

# Завантаження моделі та токенізатора
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Завантаження метрик
rouge_metric = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

# Функція для обчислення BLEU
def calculate_bleu(predictions, references):
    smoothie = SmoothingFunction().method1
    bleu_scores = []

    for pred, ref in zip(predictions, references):
        pred_tokens = nltk.word_tokenize(pred.lower())
        ref_tokens = [nltk.word_tokenize(ref.lower())]
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)
        bleu_scores.append(score)

    return np.mean(bleu_scores)

# Функція препроцесингу для T5
def preprocess_function(examples):
    # Додаємо префікс для задачі сумаризації
    prefix = "summarize: "
    inputs = [prefix + doc for doc in examples["article"]]

    # Токенізуємо статті
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Токенізуємо сумаризації
    labels = tokenizer(
        examples["highlights"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # Додаємо мітки у вхідні дані
    model_inputs["labels"] = labels["input_ids"]

    # Замінюємо padding токени на -100 (ігноруються при обчисленні втрат)
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in model_inputs["labels"]
    ]

    return model_inputs

# Застосовуємо препроцесинг до датасетів
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# Видаляємо непотрібні стовпці
columns_to_remove = ["article", "highlights", "id"]
tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_val = tokenized_val.remove_columns(columns_to_remove)

# Встановлюємо формат для PyTorch
tokenized_train.set_format("torch")
tokenized_val.set_format("torch")

# Data collator для паддінгу
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Створюємо колекцію оригінальних текстів для метрик
# Зберігаємо оригінальні сумаризації для обчислення метрик
original_summaries = val_dataset["highlights"]

# Безпечна функція для обчислення метрик з BLEU та BERTScore
def safe_compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Декодуємо згенеровані сумаризації
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Беремо відповідні оригінальні сумаризації
    batch_size = len(predictions)
    references = original_summaries[:batch_size]

    # Рахуємо ROUGE метрики
    rouge_results = rouge_metric.compute(
        predictions=decoded_preds,
        references=references,
        use_stemmer=True
    )

    # Рахуємо BERTScore
    bert_results = bertscore.compute(
        predictions=decoded_preds,
        references=references,
        lang="en"
    )

    # Рахуємо BLEU
    bleu_score = calculate_bleu(decoded_preds, references)

    # Форматуємо результати
    results = {
        "rouge1": round(rouge_results["rouge1"], 4),
        "rouge2": round(rouge_results["rouge2"], 4),
        "rougeL": round(rouge_results["rougeL"], 4),
        "bertscore_f1": round(np.mean(bert_results["f1"]), 4),
        "bleu": round(bleu_score, 4)
    }

    return results

# Створюємо кастомний колбек для обчислення метрик під час тренування
class MetricsCallback(TrainerCallback):
    def __init__(self, tokenized_eval_dataset, tokenizer, model, original_summaries, logging_steps=50):
        self.tokenized_eval_dataset = tokenized_eval_dataset
        self.tokenizer = tokenizer
        self.model = model
        self.original_summaries = original_summaries
        self.logging_steps = logging_steps
        self.best_metric = {"rougeL": 0.0}
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Додамо словник для відстеження метрик
        self.metrics_history = {
            "rouge1": [],
            "rouge2": [],
            "rougeL": [],
            "bertscore_f1": [],
            "bleu": [],
            "loss": [],
            "steps": []
        }

    def on_step_end(self, args, state, control, **kwargs):
        # Обчислюємо метрики на кожному logging_steps кроці
        if state.global_step % self.logging_steps == 0:
            # Переводимо модель в режим оцінки
            self.model.eval()

            # Створюємо DataLoader для оцінки
            from torch.utils.data import DataLoader
            eval_dataloader = DataLoader(
                self.tokenized_eval_dataset,
                batch_size=4,
                collate_fn=data_collator,
                shuffle=False
            )

            # Беремо перші 5 батчів для швидкості
            all_preds = []
            all_indices = []

            with torch.no_grad():
                for i, batch in enumerate(eval_dataloader):
                    if i >= 5:  # Обмежуємо кількість батчів
                        break

                    # Переносимо батч на пристрій (GPU/CPU)
                    batch = {k: v.to(self.device) for k, v in batch.items()}

                    # Генеруємо передбачення
                    outputs = self.model.generate(
                        input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        max_length=128
                    )

                    # Зберігаємо передбачення
                    all_preds.extend(outputs.cpu().numpy())

                    # Запам'ятовуємо індекси прикладів
                    batch_indices = list(range(i*4, min((i+1)*4, len(self.tokenized_eval_dataset))))
                    all_indices.extend(batch_indices)

            # Декодуємо передбачення
            decoded_preds = self.tokenizer.batch_decode(all_preds, skip_special_tokens=True)

            # Беремо відповідні оригінальні сумаризації
            references = [self.original_summaries[idx] for idx in all_indices]

            # Обчислюємо метрики
            rouge_results = rouge_metric.compute(
                predictions=decoded_preds,
                references=references,
                use_stemmer=True
            )

            # Обчислюємо BERTScore
            bert_results = bertscore.compute(
                predictions=decoded_preds,
                references=references,
                lang="en"
            )

            # Обчислюємо BLEU
            bleu_score = calculate_bleu(decoded_preds, references)

            # Форматуємо результати
            metrics = {
                "rouge1": round(rouge_results["rouge1"], 4),
                "rouge2": round(rouge_results["rouge2"], 4),
                "rougeL": round(rouge_results["rougeL"], 4),
                "bertscore_f1": round(np.mean(bert_results["f1"]), 4),
                "bleu": round(bleu_score, 4)
            }

            # Отримуємо втрати з логів
            loss = kwargs.get("logs", {}).get("loss", 0)

            # Зберігаємо метрики для побудови графіків
            self.metrics_history["steps"].append(state.global_step)
            for metric_name in ["rouge1", "rouge2", "rougeL", "bertscore_f1", "bleu"]:
                self.metrics_history[metric_name].append(metrics[metric_name])
            self.metrics_history["loss"].append(loss)

            # Виводимо метрики
            print(f"\nКрок {state.global_step}: ROUGE-1={metrics['rouge1']:.4f}, ROUGE-2={metrics['rouge2']:.4f}, "
                 f"ROUGE-L={metrics['rougeL']:.4f}, BERTScore={metrics['bertscore_f1']:.4f}, BLEU={metrics['bleu']:.4f}")

            # Зберігаємо найкращу модель за RougeL
            if metrics["rougeL"] > self.best_metric["rougeL"]:
                self.best_metric["rougeL"] = metrics["rougeL"]
                print(f"Новий найкращий результат! ROUGE-L: {metrics['rougeL']:.4f}")

                # Зберігаємо найкращу модель
                if args.output_dir is not None:
                    output_dir = f"{args.output_dir}/best_model"
                    self.model.save_pretrained(output_dir)
                    self.tokenizer.save_pretrained(output_dir)

            # Повертаємо модель в режим тренування
            self.model.train()

            # Логуємо в wandb
            wandb.log({
                "train/rouge1": metrics["rouge1"],
                "train/rouge2": metrics["rouge2"],
                "train/rougeL": metrics["rougeL"],
                "train/bertscore_f1": metrics["bertscore_f1"],
                "train/bleu": metrics["bleu"],
                "train/loss": loss,
                "step": state.global_step
            })

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ilya-efa (ilya-efa-winstars-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [37]:
# Аргументи для навчання
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_cnn_summarization",
    eval_steps=200,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="steps",
    save_steps=200,
    report_to="wandb",
)

# Створюємо кастомний колбек для метрик
metrics_callback = MetricsCallback(
    tokenized_eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    model=model,
    original_summaries=original_summaries,
    logging_steps=50
)

# Ініціалізація Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=safe_compute_metrics,
    data_collator=data_collator,
    callbacks=[metrics_callback],
)

In [39]:
# Оцінка метрик до файнтюнінгу
print("Оцінка метрик до файнтюнінгу...")
eval_before_results = trainer.evaluate()
print("Метрики до файнтюнінгу:")
print(eval_before_results)

Оцінка метрик до файнтюнінгу...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Метрики до файнтюнінгу:
{'eval_loss': 2.564004898071289, 'eval_model_preparation_time': 0.0192, 'eval_rouge1': 0.2491, 'eval_rouge2': 0.0854, 'eval_rougeL': 0.1991, 'eval_bertscore_f1': 0.8565, 'eval_bleu': 0.0317, 'eval_runtime': 13.5261, 'eval_samples_per_second': 7.393, 'eval_steps_per_second': 1.848}


In [41]:
# Навчання моделі
print("Початок навчання моделі...")
trainer.train()

Початок навчання моделі...


Step,Training Loss
50,2.318800
100,2.201100
150,2.197800
200,2.180400
250,2.202200
300,2.156700
350,2.127700
400,2.038100
450,2.093000
500,2.069300



Крок 50: ROUGE-1=0.2883, ROUGE-2=0.1100, ROUGE-L=0.2234, BERTScore=0.8670, BLEU=0.0774
Новий найкращий результат! ROUGE-L: 0.2234

Крок 100: ROUGE-1=0.2868, ROUGE-2=0.1158, ROUGE-L=0.2258, BERTScore=0.8663, BLEU=0.0809
Новий найкращий результат! ROUGE-L: 0.2258

Крок 150: ROUGE-1=0.2883, ROUGE-2=0.1325, ROUGE-L=0.2354, BERTScore=0.8678, BLEU=0.1082
Новий найкращий результат! ROUGE-L: 0.2354

Крок 200: ROUGE-1=0.2794, ROUGE-2=0.1135, ROUGE-L=0.2139, BERTScore=0.8656, BLEU=0.0876

Крок 250: ROUGE-1=0.2793, ROUGE-2=0.1144, ROUGE-L=0.2161, BERTScore=0.8656, BLEU=0.0886

Крок 300: ROUGE-1=0.2753, ROUGE-2=0.1244, ROUGE-L=0.2029, BERTScore=0.8632, BLEU=0.1052

Крок 350: ROUGE-1=0.3009, ROUGE-2=0.1267, ROUGE-L=0.2218, BERTScore=0.8686, BLEU=0.1017

Крок 400: ROUGE-1=0.2900, ROUGE-2=0.1245, ROUGE-L=0.2075, BERTScore=0.8648, BLEU=0.0964

Крок 450: ROUGE-1=0.2829, ROUGE-2=0.1231, ROUGE-L=0.2108, BERTScore=0.8609, BLEU=0.0983

Крок 500: ROUGE-1=0.2787, ROUGE-2=0.1251, ROUGE-L=0.2103, BERTScore=0.

TrainOutput(global_step=750, training_loss=2.127215993245443, metrics={'train_runtime': 206.2624, 'train_samples_per_second': 14.545, 'train_steps_per_second': 3.636, 'total_flos': 408190873239552.0, 'train_loss': 2.127215993245443, 'epoch': 3.0})

In [42]:
# Оцінка метрик після файнтюнінгу
print("Оцінка метрик після файнтюнінгу...")
eval_results = trainer.evaluate()
print("Метрики після файнтюнінгу:")
print(eval_results)

# Логуємо підсумкові результати в wandb
wandb.log({
    "final/rouge1": eval_results.get("eval_rouge1", 0),
    "final/rouge2": eval_results.get("eval_rouge2", 0),
    "final/rougeL": eval_results.get("eval_rougeL", 0),
    "final/bertscore_f1": eval_results.get("eval_bertscore_f1", 0),
    "final/bleu": eval_results.get("eval_bleu", 0)
})

# Зберігаємо фінальну модель
trainer.save_model("./t5_cnn_summarization_final")

Оцінка метрик після файнтюнінгу...


Метрики після файнтюнінгу:
{'eval_loss': 2.1034140586853027, 'eval_model_preparation_time': 0.0192, 'eval_rouge1': 0.243, 'eval_rouge2': 0.0887, 'eval_rougeL': 0.1971, 'eval_bertscore_f1': 0.8598, 'eval_bleu': 0.0262, 'eval_runtime': 13.6365, 'eval_samples_per_second': 7.333, 'eval_steps_per_second': 1.833, 'epoch': 3.0}


In [53]:
for item_before, item_after in zip(eval_before_results.items(), eval_results.items()):
  print(f'{item_before[0]}')
  print(f'before fine-tune: {item_before[1]}')
  print(f'after fine-tune: {item_after[1]}')
  print('--------')

eval_loss
before fine-tune: 2.564004898071289
after fine-tune: 2.1034140586853027
--------
eval_model_preparation_time
before fine-tune: 0.0192
after fine-tune: 0.0192
--------
eval_rouge1
before fine-tune: 0.2491
after fine-tune: 0.243
--------
eval_rouge2
before fine-tune: 0.0854
after fine-tune: 0.0887
--------
eval_rougeL
before fine-tune: 0.1991
after fine-tune: 0.1971
--------
eval_bertscore_f1
before fine-tune: 0.8565
after fine-tune: 0.8598
--------
eval_bleu
before fine-tune: 0.0317
after fine-tune: 0.0262
--------
eval_runtime
before fine-tune: 13.5261
after fine-tune: 13.6365
--------
eval_samples_per_second
before fine-tune: 7.393
after fine-tune: 7.333
--------
eval_steps_per_second
before fine-tune: 1.848
after fine-tune: 1.833
--------


###### До файн-тюну

In [110]:
few_shot_results[0]['article']

'(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don\'t know, but the fact that so many people can have a life extension, that\'s pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I\'m just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard\'s gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, according to California Pacific Medical Ce

In [108]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_text = few_shot_results[0]['article']

# Prefix as used during preprocessing
prefix = "summarize: "
prompt = prefix + input_text

# Tokenize the prompt
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512,  # Must match training
    padding="max_length"
).to(model.device)

# Generate the summary
model.eval()
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=150,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )

# Decode the summary
summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the result
print("Summary:")
print(summary)


Summary:
the power that multiplied Zully Broussard's gift was data processing of genetic profiles from donor-recipient pairs. it is taking five surgeons, a covey of physician assistants, nurses and anesthesiologists to perform surgeries on 12 people. the chain of surgeries is to be wrapped up Friday.


###### Після файн-тюну

In [109]:
model_name = "/content/t5_cnn_summarization/checkpoint-750"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_text = few_shot_results[0]['article']

# Prefix as used during preprocessing
prefix = "summarize: "
prompt = prefix + input_text

# Tokenize the prompt
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512,  # Must match training
    padding="max_length"
).to(model.device)

# Generate the summary
model.eval()
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=150,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )

# Decode the summary
summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the result
print("Summary:")
print(summary)


Summary:
"I thought I was going to help this one person who I don't know," says Zully Broussard. The power that multiplied Broussard's gift was data processing of genetic profiles from donor-recipient pairs. In late March, the medical center is planning to hold a reception for all 12 patients.
